# Scraping A Large Dataset of Foods

In [ ]:
''' 
So currently, this function isn't optimal
- Issues with the dataframe, where the column titles we want in the main dataframe are all stored in a column and
  values are stored in the column next to it
- I guess we could look at the 100g/100ml column, or the serving size column(just need to drop some columns on the end of scraped df)
- Need to figure out which nutrient rows to keep and which to get rid of, not all food items have the same things
  - Could just add a 0 or NULL value if it doesn't exist, but then normalizing would get weird because we have to drop NaN values
- I completely forgot to scrape the nutrition score letter
'''



def extract_nutrition(url):
    """ takes href input to pull out the nutrition information

    Args:
        url (str): link to pull nutrition information from

    Output:
        nutr_table (dataframe): something to add onto the row of the original dataframe
    """
    # get all tables
    list_of_tables = pd.read_html(url)
    
    # select table of nutrients
    table = list_of_tables[0]

    # process table
    table = table[table[0:2]]

    nutr_table = table.set_index('Nutrition facts')

    nutr_table = nutr_table.transpose()

    nutr_table = nutr_table.reset_index(drop=True)

    return

In [126]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

def get_food_links(url):
    '''Takes a url and returns a dataframe with the food names and links to the food pages
    
    Args:
        url (str): url to scrape

    Returns:
        foods (dataframe): dataframe with food names and links to the food pages
    '''

    # Get the data
    r = requests.get(url)
    # soup response
    soup = BeautifulSoup(r.text, "html.parser")
    # Create a dataframe
    foods = pd.DataFrame(columns=["name", "url"])
    foods = foods.set_index("name")

    # Get unordered list of foods
    ul = soup.find_all('ul')[10]
    # Get the links from the foods
    a = ul.find_all('a')
    for i in a:
        # extract and add their title and href to the dataframe
        foods.loc[i.get("title")] = "https://us.openfoodfacts.org"+i.get("href")
        

    return foods

In [127]:
df = pd.DataFrame(columns=["name", "url"])
df = df.set_index("name")

for i in range(1, 31):
    url = "https://us.openfoodfacts.org/" + str(i)
    df = pd.concat([df, get_food_links(url)])

df

,url
name,
Nutella - Ferrero - 400 g,https://us.openfoodfacts.org/product/301762042...
Prince - Lu - 300 g,https://us.openfoodfacts.org/product/762221044...
Nutella - Ferrero - 1 kg,https://us.openfoodfacts.org/product/301762042...
Coca Cola Zero - 330 ml,https://us.openfoodfacts.org/product/544900013...
"Muesli Raisin, Figue, Abricot - Bjorg - 375 g",https://us.openfoodfacts.org/product/322982012...
...,...
Kosher dill baby wholes - Vlasic,https://us.openfoodfacts.org/product/005410000...
"Vlasic, stackers, bread & butter pickles",https://us.openfoodfacts.org/product/005410000...
Pickle relish - Vlasic,https://us.openfoodfacts.org/product/005410001...
